* In this repo we focus on the **SMO algorithm** (see "summary.pdf" Step5).

It is one of the most well-known algorithms for **training Support Vector Machines**.

* SMO works in an iterative manner. At first the values of a and b are initialized. In every iteration we select two indices i, j
and solve the dual problem ("summary.pdf" Step4) by fixing all a's for indices diffirent to i and j. 
An iterative step for a selected pair of indices i,j is described in the "**Step**" function below.

**Remark** In this repo we choose the **iteration step** indices as suggested in well-known paper of John Platt "Sequential Minimal Optimization A Fast Algorithm for training Support Vector Machines" in pages 8-9.

* We develop two variations based on the type of boundary we want to get; either **linear** or **gaussian kernel** boundary.

In [1]:
import numpy as np

In [2]:
class SMO():
    def __init__(self, X,y,steps,C, kernel = None, sigma = None ) :
        """
        X : is an np.array matrix where each row corresponds to a dataset point
        y : is an np.array vector with the labels of the datapoints
        steps : describes the number of iterations in the SMO algorithm
        C : is the optimal margin's hyperparameter that controls the width of the margin area and the number of misclassifications
        kernel : if None then we get the parameters of the Optimal Margin Linear Boundary
                 if not None then we get the parameters of the Optimal Margin Gaussian Kernel Boundary
        sigma : the hyperparameter of the Gaussian kernel
        """
        self.X = X
        self.y = y
        self.steps = steps
        self.C = C
        self.kernel = kernel
        self.sigma = sigma
        if self.kernel != None and self.sigma == None :
            print('Invalid arguments! Either remove the kernel attribute or insert the kernel sigma hyperparameter.')
    
    def Gauss(self, t, x):
        """
        t : is a dataset vector in an np.array form
        x : is either a dataset vector or a bunch of vectors
        The output ie either a number or a vector
        """
        if self.sigma == None :
            return 'no sigma kernel hyperparameter was given as an attribute'
        else :
            if t.size == x.size : 
                return 1/np.exp( np.sum((t-x)**2) / (2*self.sigma**2) ) #number
            else : #else in case x is the bunch of all datapoints
                res = np.array([])
                for j in range(x.shape[0]) :
                    res = np.append(res, 1/np.exp( np.sum((t-x[j])**2) / (2*self.sigma**2) ))
                return res #vector

    def Bndry(self,t,a,b):
        """
        t : is either an 1-dim np.array vector OR a 2-dim np.array vector OR a 2-dim np.array matrix
        Computes the boundary function at point t (see a and b dependent formula in summary notes)
        """
        if self.kernel == None :
            if t.ndim == 1 :
                return np.sum(a*self.y*(self.X@t)) + b #number
            if t.ndim == 2:
                if t.size == self.X[0].size : # ie if t is a vector
                    return np.sum(a*self.y*(self.X@(t.T))) + b #number
                else : # ie if t is a matrix
                    res = np.array([])
                    for j in range(t.shape[0]):
                        res = np.append( res, np.sum(a*self.y*(self.X@t[j])) + b )
                    return res #vector
        elif self.kernel != None :
            if t.ndim == 1 :
                return np.sum(a*self.y*self.Gauss(t,self.X)) + b
            if t.ndim == 2 :
                if t.size == self.X[0].size :
                    return np.sum(a*self.y*(self.Gauss(t,self.X))) + b
                else :
                    res = np.array([])
                    for j in range(t.shape[0]):
                        res = np.append(res, np.sum(a*self.y*(self.Gauss(t[j],self.X))) + b)
                    return res
        
    
    def Step(self,i,j,a,b):
        """
        i, j : are dataset point indices
        Outputs the optimized a and b after an SMO iteration
        """

        if self.kernel==None :
            delta = self.y[i]*((self.Bndry(self.X[j,:],a,b)-self.y[j]) - (self.Bndry(self.X[i,:],a,b)-self.y[i]))
            Chi = self.X[i,:].dot(self.X[i,:]) + self.X[j,:].dot(self.X[j,:]) - 2*self.X[i,:].dot(self.X[j,:])
        else :
            delta = self.y[i]*((self.Bndry(self.X[j,:],a,b)-self.y[j])-(self.Bndry(self.X[i,:],a,b)-self.y[i]))
            Chi = self.Gauss(self.X[i,:],self.X[i,:]) + self.Gauss(self.X[j,:],self.X[j,:]) -2*self.Gauss(self.X[i,:],self.X[j,:]) 

        s = self.y[i]*self.y[j]
        gamma = s*a[i]+a[j]

        if s == 1:
            L = max(0,gamma-C); H = min(gamma,C)
        else:
            L = max(0,-gamma); H = min(C,C-gamma)

        if Chi > 0:
            a[i] = min(max(a[i]+delta/Chi,L),H)
        elif delta > 0:
            a[i] = L
        else:
            a[i] = H

        a[j] = gamma-s*a[i]

        if self.kernel==None :
            b = b-1/2*(self.Bndry(self.X[i,:],a,b)-self.y[i]+self.Bndry(self.X[j,:],a,b)-self.y[j])
        else :
            b = b-1/2*(self.Bndry(self.X[i,:],a,b)-self.y[i]+self.Bndry(self.X[j,:],a,b)-self.y[j])

        return a,b

    
    
    def Smo(self):
        """
        Outputs the optimal support vectors a and the optimal b.
        These two parameters characterize the desired optimal margin boundary in both linear and kernelized cases
        """
        a = np.zeros(self.X.shape[0])
        b = 0
        kkt = np.ones(a.size)
        count = 0
        while (count < self.steps):
            for i in range(a.size): 
                H_i = self.Bndry(X[i,:],a,b)  
                kkt[i] = (self.C-a[i])*max(0,1-self.y[i]*H_i) + a[i]*max(0,self.y[i]*H_i-1)
                if kkt[i]>0:
                    count += 1
                    # take an array of all possible indices, erase all that aren't margin vectors and erase i from the list too
                    k = np.arange(self.X.shape[0]) 
                    t = k[(a>0) & (a<self.C)]
                    t = np.delete(t,np.where( t == i))
                    # if there are indices which fullfill this restriction pick one of them
                    if t.size > 0: 
                        j = np.random.choice(t)
                        temp = self.Step(i,j,a,b)
                        betha = temp[0]
                        b = temp[1]
                    # else just pick a random index of all possible but i
                    else:
                        k = np.delete(k,np.where(k == i))
                        j = np.random.choice(k)
                        temp = self.Step(i,j,a,b)
                        betha = temp[0]
                        b = temp[1]
            # check if all  KKT_i are zero, if so stop
            if np.array_equal(kkt, np.zeros(kkt.size)):
                break

        med = np.zeros(self.X.shape[0])
        supcount = 0
        for i in range(self.X.shape[0]):
            if a[i] > 0:
                med[i] = self.Bndry(self.X[i,:],a,b) - self.y[i]
                supcount += 1 
        averg = np.sum(med)/supcount
        b = b - averg

        return a,b
